# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '49fb0c68-cbc5-4e1f-ab26-e81c61e377be', 'created_at': '2025-08-14T08:20:34.227633+00:00', 'updated_at': '2025-08-14T08:20:34.227633+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
49fb0c68-cbc5-4e1f-ab26-e81c61e377be


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [ ]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 요약 - 보고서 전체의 핵심 발견과 주요 수치(시장규모, 성장률, 주요 이슈)를 간결히 정리하여 의사결정자가 빠르게 파악할 수 있도록 제공. 
3. 시장 규모 및 경쟁 지표 - 최근 연도(2022~2024)를 중심으로 한국 반도체 시장 규모, 제품별(메모리·비메모리·파운드리) 매출 비중, 글로벌 점유율 및 주요 경쟁국 비교 지표를 정리. 
4. 기술 동향 및 R&D 현황 - D램·낸드·HBM 등 메모리 기술 진화와 EUV·GAA 등 첨단 공정 채택 현황, 파운드리 로드맵, 국내 기업의 R&D 투자 규모와 주요 연구주제 분석. 
5. 공급망·수출 동향과 리스크 - 장비·소재의 수입 의존도, 주요 수출국별 비중(중국·미국·유럽 등), 최근 수출 규제 및 통상 리스크, 공급망 병목 사례와 대응 현황을 정리. 
6. 정부 정책·지원 제도 - 국가 차원의 반도체 육성 정책, 세제·보조금·토지·인프라 지원, 반도체 특별법 및 규제 완화 현황과 최근 발표된 정책(2023~2025)의 영향 분석. 
7. 주요 기업 분석 (삼성·SK·팹리스·파운드리) - 삼성전자·SK하이닉스 등 대기업의 생산능력·투자계획·제품포트폴리오와 팹리스·파운드리·장비·소재 기업들의 경쟁력 및 생태계 역할을 기업별로 비교 분석. 
8. 미래 전망 및 시나리오 - 글로벌 수요전망, 기술 전환(3nm→2nm·GAA·칩렛 등)에 따른 영향, 정책·무역 리스크별 시나리오(낙관·기준·비관)와 시사점을 제시. 
9. 결론 및 권고사항 - 보고서 핵심 결론을 정리하고, 산업 경쟁력 강화와 리스크 완화를 위한 정책·기업 차원의 실무적 권고사항을 제시. 

##########  AI 메시지  

In [ ]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [ ]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

In [ ]:
len(state["values"]["slides"])

In [ ]:
state["values"]["slides"]

In [ ]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])